In [ ]:
#根据给定的参数，使用AutoConfig,AutoModelForpretrain,等映射机制，实例化流程组件
#task,model参数任意选择
#返回AutoPipeline的
from transformers import pipeline

In [ ]:
pipe = pipeline(task='name',model='name')

# 基类PIPELINE
    就是推断模型的类，对于不同的任务都要实现一个特定的子类，因为不同任务预处理和后处理方法不同
    子类需要实现以下三个方法
    preprocess：预处理输入[str,PIL.image]
        inPut处理，一个样本，载入或读取
        需要调用image_processor，
        最后返回字典，要生成对应的键值对{'image':input}
    _forward
        从上面的结果取出输入
        调用model(**input)
        返回模型特定输出类
    postprocess：
        后处理模型输出类，
        可能会再次调用image_processor.post_process函数
        解码生成annotation 字典
    使用pipeline，会调用__call__方法，所以是直接当作模型使用即可  

In [ ]:
#也可以将pipeline拆开使用

In [ ]:
from transformers import AutoImageProcessor, AutoModelForDepthEstimation

checkpoint = "vinvino02/glpn-nyu"

image_processor = AutoImageProcessor.from_pretrained(checkpoint)
model = AutoModelForDepthEstimation.from_pretrained(checkpoint)

In [ ]:
import torch

with torch.no_grad():
    outputs = model(pixel_values)
    predicted_depth = outputs.predicted_depth

In [ ]:
import numpy as np

# interpolate to original size
prediction = torch.nn.functional.interpolate(
    predicted_depth.unsqueeze(1),
    size=image.size[::-1],
    mode="bicubic",
    align_corners=False,
).squeeze()
output = prediction.numpy()

formatted = (output * 255 / np.max(output)).astype("uint8")
depth = Image.fromarray(formatted)